# Part 1: Camera calibration (Intrinsic parameters)

In [ ]:
import numpy as np
import cv2
import glob

In [ ]:
 # termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

In [ ]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.


In [ ]:
images = glob.glob('./Chessboard/*.JPG')


In [ ]:
print(images)

In [ ]:
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (9,6), corners,ret)
        cv2.imwrite('./result/part1/'+str(fname),img)
        cv2.imshow('img',img)
        cv2.waitKey(500)
cv2.destroyAllWindows()

In [ ]:
 ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

In [ ]:
print(ret)


In [ ]:
print(mtx)

In [ ]:
principalPoint = (mtx[0][2],mtx[1][2])

In [ ]:
focal = (mtx[0][0]+mtx[1][1])/2

In [ ]:
print(principalPoint)

# Part 2: Take the pictures

# Part 3: Compute the relative camera pose

In [ ]:
from matplotlib import pyplot as plt

img1 = cv2.imread('./Pair image/myleft.JPG',0)  #queryimage # left image
img2 = cv2.imread('./Pair image/myright.JPG',0) #trainimage # right image


In [ ]:
cv2.imshow('img1',img1)
#cv2.waitKey(500)
cv2.destroyAllWindows()


In [ ]:
import cv2
cv2.__version__

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# FLANN parameters
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)

good = []
pts1 = []
pts2 = []

# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        good.append(m)
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)

Got the list of best matches from both images

In [ ]:
pts1 = np.int32(pts1)
pts2 = np.int32(pts2)
F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)

# We select only inlier points
pts1 = pts1[mask.ravel()==1]
pts2 = pts2[mask.ravel()==1]

In [ ]:
print(F)

Next we find the epilines. Epilines corresponding to the points in first image is drawn on second image. So mentioning of correct images are important here. We get an array of lines. So we define a new function to draw these lines on the images.

In [ ]:
def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c = img1.shape
    img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

Now we find the epilines in both the images and draw them.

In [ ]:
# Find epilines corresponding to points in right image (second image) and
# drawing its lines on left image
lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(img1,img2,lines1,pts1,pts2)

# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(img2,img1,lines2,pts2,pts1)

plt.subplot(121),plt.imshow(img5)
plt.subplot(122),plt.imshow(img3)
plt.show()
cv2.destroyAllWindows()

Compute the essential matrix E
https://docs.opencv.org/3.1.0/d9/d0c/group__calib3d.html#ga13f7e34de8fa516a686a56af1196247f

In [ ]:
E, mask = cv2.findEssentialMat(pts1, pts2, focal, principalPoint, method=cv2.RANSAC, prob=0.999, threshold=3.0)

In [ ]:
print(E)

In [ ]:
R1,R2,t = cv2.decomposeEssentialMat(E)

### decompose the essential matrix, we got two rotaion matrixs, R1(3*3) and R2(3*3), and one translation vector (3*1).
### We need the  projection matrix of the first camera and projection matrix of the second camera to calculate the triangulatePoints
### projMatr1 – 3x4 projection matrix of the first camera.
### projMatr2 – 3x4 projection matrix of the second camera.

In [ ]:
print(R1)

In [ ]:
print(R2)

In [ ]:
print(t)

### There are 4 possible combination for the projection matrix.
#### Case 1:  projMatr1 = K [ I 0]   projMatr2 = K[R1 t]
#### Case 2:  projMatr1 = K[ I 0]   projMatr2 = K[R1  -t]
#### Case 3:  projMatr1 = K[ I 0]   projMatr2 = K[R2 t]
#### Case 4:  projMatr1 = K[ I 0]   projMatr2 = K[R2 -t]


In [ ]:
points1 = np.array(pts1).astype(float).T 
points2 = np.array(pts2).astype(float).T

In [ ]:
# make a projection matrix for camera 1
projMatr1 = mtx@np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0]])

In [ ]:
# case 1
projMatr21 = mtx@np.concatenate((R1,t),1)
# case 2
projMatr22 = mtx@np.concatenate((R1,-t),1)
# case 3
projMatr23 = mtx@np.concatenate((R2,t),1)
# case 4
projMatr24 = mtx@np.concatenate((R2,-t),1)

In [ ]:
triangulate1 = cv2.triangulatePoints(projMatr1, projMatr21,points1,points2)
triangulate2 = cv2.triangulatePoints(projMatr1, projMatr22,points1,points2)
triangulate3 = cv2.triangulatePoints(projMatr1, projMatr23,points1,points2)
triangulate4 = cv2.triangulatePoints(projMatr1, projMatr24,points1,points2)

In [ ]:
print(triangulate1.T)

#### case 1 got negtive distance, get rid of it.

In [ ]:
print(triangulate2.T)

#### case 2 got negtive distance, get rid of it.

In [ ]:
print(triangulate3.T)

#### case 3 got negtive distance, so I get rit of this case.

In [ ]:
print(triangulate4.T)

#### case 4 has the positive distance , so the (R2,-t) is the right R and t

In [ ]:
triangulate4.T

In [ ]:
triangulate4world = np.true_divide(triangulate4.T[:,:3], triangulate4.T[:,[-1]]).T

In [ ]:
print(triangulate4world.T)

In [ ]:
print(triangulate4world.shape)

In [ ]:
pixelh = mtx@triangulate4world

In [ ]:
print(pixelh.T)

In [ ]:
pixel = np.true_divide(pixelh.T[:,:2],pixelh.T[:,[-1]])

In [ ]:
print(points1.T)

In [ ]:
print(pixel)

In [ ]:
print(pixel.shape)

In [ ]:
imgleft = cv2.imread('./Pair image/myleft.JPG')  #queryimage # left image
imgright = cv2.imread('./Pair image/myright.JPG') #trainimage # right image



In [ ]:
cv2.imshow('left',imgleft)

In [ ]:
for point in pixel.astype(int):
    #print(tuple(point))
    
    imgleft = cv2.circle(imgleft,tuple(point),10,(0, 0, 255),-1)
for point in points1.T.astype(int):
    #print(tuple(point))
    imgleft = cv2.circle(imgleft,tuple(point),5,(0, 255, 0),-1)
cv2.imshow('left',imgleft )
cv2.waitKey(0)
cv2.destroyAllWindows()
#cv2.imwrite('point',img13)


# Part 4: Plane-sweeping stereo

In [ ]:
imgL = cv2.imread('./Pair image/myleft.JPG',0)
imgR = cv2.imread('./Pair image/myright.JPG',0)
stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)
disparity = stereo.compute(imgL,imgR)
plt.imshow(disparity,'gray')
plt.show()